<a href="https://colab.research.google.com/github/jsaroni/QHack2023_Feynmanprodigies/blob/main/final_submission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

QHack2022 Open Hackathon - FEYNMAN PRODIGIES TEAM




Project Name: Efficient ground state computation methodologies for BeH2




In this notebook, we describe the implementation of different methodologies to find the ground state of the molecule BeH2, each with their own merits and drawbacks. The methods used consider only single and double excitations. Some of the methodologies that we implement greatly simplify the model while sacrificing chemical accuracy while others use error mitigation more efficiently through qiskit runtime which works well with hybrid algorithms such as VQE.


The first method is based on a crude approximation, restricting the Hilbert space to an active space defined by unbonded electrons within the BeH2 molecule.


We are trying to find the ground state of BeH2, so it is worth knowing more on its physical chemistry. Beryllium has chemical formula 1s2 2s2 and the two hydrogens 1s1 in electron configuration notation. When the molecule BeH2 is formed, the s orbitals from the two hydrogens hybridize with the p orbitals from beryllium to form two sp molecular orbital bonds that want to be as far away from each other as possible due to electron repulsion and this results in a linear molecule. We use the sto3g basis to approximate slater type orbitals.

The following module need installation: 
qiskit for testing our quantum simulations.
pyscf as an open source collection of electronic structure modules.
qiskit_nature.
qiskit_ibm_runtime.

In [ ]:
!pip install qiskit
!pip install qiskit_nature
!pip install pyscf
!pip install qiskit_ibm_runtime

In [ ]:
# Import necessary libraries and packages
import math
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')
from qiskit import Aer, IBMQ, QuantumCircuit
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Options, Sampler, Estimator
from qiskit.providers.aer import StatevectorSimulator
from qiskit.utils import QuantumInstance
from qiskit.providers.fake_provider import FakeGuadalupe, FakeManila
from qiskit_aer.noise import NoiseModel
from qiskit.tools.jupyter import *
from qiskit.visualization import *

# Import Qiskit libraries for VQE
from qiskit.algorithms import MinimumEigensolverResult, VQE
from qiskit.algorithms.optimizers import SLSQP, SPSA, COBYLA

# Import Qiskit Nature libraries
from qiskit_nature.algorithms import GroundStateEigensolver, VQEUCCFactory
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.circuit.library import UCC, UCCSD
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureDriverType, ElectronicStructureMoleculeDriver
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import BravyiKitaevMapper, JordanWignerMapper, ParityMapper
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer

# Prototype-zne
!pip install prototype-zne --quiet

from qiskit_nature.settings import settings

settings.dict_aux_operators = True

We use the linear geometry of the molecule and set a distance d as rougly the equilibrium distance of the two hydrogen atoms from beryllium which is 1.29 Angstroms.

In [ ]:
#### Fill in the values below and complete the function to define the Molecule ####
d = 1.29
# Coordinates are given in Angstrom
# Replace hydrogen_t with appropriate molecule
hydrogen_t = [["Be", [0, 0.0, 0.0]], # ----------- Enter your code here
              ["H", [-d, 0.0, 0.0]], # ----------- Enter your code here
              ["H", [d, 0.0, 0.0]]]
                  
h3p = Molecule( # Fill up the function below
    geometry= hydrogen_t,  # ----------- Enter your code here
    multiplicity= 1, # ----------- Enter your code here
    charge= 0, # ----------- Enter your code here
)

driver = ElectronicStructureMoleculeDriver(h3p, basis="sto3g", driver_type=ElectronicStructureDriverType.PYSCF) 

properties = driver.run()

The number of electrons in the molecule is 6 as expected.

In [ ]:
num_alpha_electrons = properties.get_property('ParticleNumber').num_alpha
num_beta_electrons = properties.get_property('ParticleNumber').num_beta
num_spin_orbitals = int(properties.get_property('ParticleNumber').num_spin_orbitals)

nuclear_rep_energy = properties.get_property('ElectronicEnergy').nuclear_repulsion_energy
print("number of alpha electrons: " , num_alpha_electrons)
print("number of beta electrons: " , num_beta_electrons)
print("number of spin orbitals: " , num_spin_orbitals)
print("nuclear repulsion energy: " , nuclear_rep_energy)

In [ ]:
# Check the occupation of the spin orbitals
PN_property = properties.get_property("ParticleNumber")

# Define the active space around the Fermi level 
# (selected automatically around the HOMO and LUMO, ordered by energy)
transformer = ActiveSpaceTransformer(
    num_electrons=2, # Number of electrons in our active space
    num_molecular_orbitals=2, # Number of orbitals in our active space
)

# Now you can get the reduced electronic structure problem
problem_reduced = ElectronicStructureProblem(driver, transformers=[transformer]) 

# The second quantized Hamiltonian of the reduce problem
second_q_ops_reduced = problem_reduced.second_q_ops()

In [ ]:
# Setup the mapper and qubit converter
mapper_type = 'ParityMapper'

if mapper_type == 'ParityMapper':
    mapper = ParityMapper()
elif mapper_type == 'JordanWignerMapper':
    mapper = JordanWignerMapper()
elif mapper_type == 'BravyiKitaevMapper':
    mapper = BravyiKitaevMapper()


converter = QubitConverter(mapper)

qubit_op = converter.convert(second_q_ops_reduced["ElectronicEnergy"])
print(qubit_op)

In [ ]:
# Set the mapper to qubits
parity_mapper = ParityMapper() # This is the example of parity mapping

# Set the qubit converter with two qubit reduction to reduce the computational cost 
parity_converter = QubitConverter(parity_mapper, two_qubit_reduction=True)    

# Compute the Hamitonian in qubit form
qubit_op_parity = parity_converter.convert(second_q_ops_reduced.get('ElectronicEnergy'), num_particles=problem_reduced.num_particles)

print(qubit_op_parity)

In [ ]:
vqe_factory = VQEUCCFactory( # This is an example of UCC"SD" ansatz
    quantum_instance=Aer.get_backend("aer_simulator_statevector"),
    optimizer=SLSQP(),
    ansatz=UCC(excitations='sd')
) 

from qiskit.algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()

solver = GroundStateEigensolver(parity_converter, vqe_factory)  # Define Numpy
real_solution_t = solver.solve(problem_reduced).total_energies[0]    
print('Reference energy : ', real_solution_t)

In [ ]:
# Define our 'ansatz' for the problem
ansatz = UCC(
    qubit_converter=parity_converter,
    num_particles=problem_reduced.num_particles, 
    num_spin_orbitals=problem_reduced.num_spin_orbitals,
    excitations='sd'
)

In [ ]:
IBMQ.save_account('1f470c8fb158d77fb7a8082fb0c4e6d289a97765a76aee8b2de5eaf74d7c8839545bed73ded44e885edd5ee98205f244d9d770de26eea55277e2c740e62aa591')
service = QiskitRuntimeService()

In [ ]:
backend = service.backends(simulator=True)[0]
noisy_sim = FakeGuadalupe()
#for running on real quantum device
#backend = 'ibmq_guadalupe'


print(backend)

In [ ]:
service = QiskitRuntimeService(
    channel='ibm_quantum',
    instance='qhack-event/main/level-2-team-1',
)

In [ ]:
noise_model = NoiseModel.from_backend(noisy_sim)

options_with_em = Options(
    simulator={
        "noise_model": noise_model,
        "seed_simulator": 42,
    },  
    resilience_level=1 # You may change the value here. resilience_level = 1 will activate TREX
)


#for running on real quantum device
#options_with_em = Options(resilience_level=1 # You may change the value here. resilience_level = 1 will activate TREX
#)

In [ ]:
%%time
from qiskit.utils import algorithm_globals
algorithm_globals.random_seed = 1024

# Define convergence list
convergence = []

# Keep track of jobs (Do-not-modify)
job_list = []

# Initialize estimator object
#estimator = Estimator()# Enter your code here

# Define evaluate_expectation function
def evaluate_expectation(x):
    x = list(x)
    
    # Define estimator run parameters
    #### enter your code below ####



    with Session(service=service, backend=backend):
      estimator = Estimator(options=options_with_em)
      job = estimator.run(circuits=[ansatz], parameter_values=[x],observables=qubit_op_parity).result()
      results = job.values[0]
      job_list.append(job)


    # Pass results back to callback function
    return np.real(results)



# Call back function
def callback(x,fx,ax,tx,nx):
    # Callback function to get a view on internal states and statistics of the optimizer for visualization
    convergence.append(evaluate_expectation(fx))

np.random.seed(10)

# Define initial point. We shall define a random point here based on the number of parameters in our ansatz
initial_point = np.random.random(ansatz.num_parameters)

#### enter your code below ####
# Define optimizer and pass callback function
optimizer = SPSA(maxiter = 50, callback=callback) # ----------- Enter your code here

# Define minimize function
result =  optimizer.minimize(evaluate_expectation, x0=initial_point) # ----------- Enter your code here

In [ ]:
Energy_H_t = []
for i in range(len(convergence)):
    sol = MinimumEigensolverResult()
    sol.eigenvalue = convergence[i]
    sol = problem_reduced.interpret(sol).total_energies[0]
    Energy_H_t.append(sol)
print("Computed Energy:", Energy_H_t[-1])

In [ ]:
# The following plot compares the two Estimators - with and without noise

plt.rcParams["font.size"] = 14

# plot loss and reference value
plt.figure(figsize=(12, 6), facecolor='white')
plt.plot(Energy_H_t, label="Estimator VQE BeH2+NOISY+TREX")
plt.axhline(y=real_solution_t.real, color="tab:red", ls="--", label="Target")

plt.legend(loc="best")
plt.xlabel("Iteration")
plt.ylabel("Energy [H]")
plt.title("VQE energy")
plt.show()

print(Energy_H_t)
print(real_solution_t.real)

This can also be done using estimator from qiskit terra speedily with a noisy backend and zero-noise extrapolation. Additionally we do a classical benchmark of our approximation method of the BeH2 ground state for different distances to compare with experimental results.

The second method is based on orbital freezing.


The third method is based on the adaptive variational quantum eigensolver with Pennylane.


The last method is based on entanglement forging from circuit knitting.